#Importing Libraries

In [ ]:
import random as rnd
import time
import math
import numpy as np
from tabulate import tabulate

Generating Stream of Random Numbers

In [ ]:
#@title Randomly Generating Data Stream

with open("numbers", 'w') as f:
  no_of_element = rnd.randint(900000000, 1000000000)
  for i in range(no_of_element):
    r = rnd.randint(1000, 10000)
    f.write("{0}\n".format(rnd.randint(1, r)))


#Google Drive Authentication

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Streaming

In [ ]:
def summary(count,unique_data_dict,n_estimated_morris,n_estimated_fm):
  data = [[count,n_estimated_morris, len(unique_data_dict),n_estimated_fm]] 

  print(tabulate(data, headers=["Exact Element Count","Approximate Element Count using Morris", "Exact Distinct Element Count","Approximate Distinct elements count using FM"]))
  print()


  print("Frequency of each distinct Element in the stream {'num':count}")
  print(unique_data_dict)
  print()

In [ ]:
def hashFunction(x):
  a = 7573
  b = 9537
  c = 9975  
  hash = (a*x+b)%c
  return hash

Hash Functions for Count Min and Count Frequency

In [ ]:
def hash1(x):   
  hash = ((7173*x+3013)%10993)%10000
  return hash

def hash2(x):
  hash = ((783*x+639)%10993)%10000
  return hash

def hash3(x):
  hash = ((8124*x+8276)%10993)%10000
  return hash

def sign_hash1(x): 
  if(x%2==0):
    return 1  
  else:
    return -1

def sign_hash2(x):
  if(x%4==0):
    return 1  
  else:
    return -1

def sign_hash3(x):
  if(x%5==0):
    return 1  
  else:
    return -1

In [ ]:
def count_summary(unique_data_count,hash_table,count_hash_table):
  freq_count_min={}
  freq_count={}

  s=[]
  for each in unique_data_count:
    value1= hash1(int(each))
    value2 = hash2(int(each))
    value3 = hash3(int(each))
    freq = int(min([hash_table[0][value1],hash_table[1][value2],hash_table[2][value3]]))
    freq_count_min[each]=freq

    sign1 = sign_hash1(int(each))
    sign2 = sign_hash2(int(each))
    sign3 = sign_hash3(int(each))
    lst = [sign1*count_hash_table[0][value1],sign2*count_hash_table[1][value2],sign2*count_hash_table[2][value3]]
    sorted(lst)

    #Getting median of the values at the hash table
    freq1 = int(lst[1])
    freq_count[each]=freq1
    s.append([int(each),unique_data_count[each],freq_count_min[each],freq_count[each]])

  print("Frequency of each distinct Element in the stream using Count min {'num':count}: ")
  print(freq_count_min)
  print("Frequency of each distinct Element in the stream using Count {'num':count}: ")
  print(freq_count)

  #print(tabulate(s, headers=["Element","Exact Frequency", "Frequency calculated using Count min","Frequency calculated using Count"]))



In [ ]:
file = open("/content/drive/MyDrive/My Documents/ABD/Assignment2/numbers",'r')
count = 0
line = file.readline()  

#Keys: Unique data, value = count
unique_data_count={}
hash_count = 3

hash_table = np.zeros((hash_count,10000))
count_hash_table = np.zeros((hash_count,10000))

temp_count = 1
X=0
trailing_zeros = 0

while line:

  #Exact
  count = count+1
  line = line.rstrip() 

  if (unique_data_count.get(line) == None):
    unique_data_count[line] = 1
  else:
    unique_data_count[line] = int(unique_data_count.get(line))+1
  #End Exact

  #Morris
  value = math.pow(2,X)
  probability = 1.0/value
  if rnd.uniform(0, 1)<probability:
      X+=1
  n_estimated_morris = int(math.pow(2,X))-1
  #End Morris

  #FM
  hash_value = hashFunction(int(line))
  bin_number = bin(hash_value)[2:]

  str_bin = str(bin_number)
  t_zero  = len(str_bin)-len(str_bin.strip('0'))
  trailing_zeros = max(trailing_zeros,t_zero)
  n_estimated_fm = pow(2,trailing_zeros)
  #End FM


  #Count min and  Sketch
  #getting has value for each element in stream
  value1= hash1(int(line))
  value2 = hash2(int(line))
  value3 = hash3(int(line))

  sign1 = sign_hash1(int(line))
  sign2 = sign_hash2(int(line))
  sign3 = sign_hash3(int(line))

  #Update Hash Table
  hash_table[0][value1] = hash_table[0][value1]+1
  hash_table[1][value2] = hash_table[1][value2]+1
  hash_table[2][value3] = hash_table[2][value3]+1

  #Update Count Hash Table
  count_hash_table[0][value1] = count_hash_table[0][value1]+(sign1*1)
  count_hash_table[1][value2] = count_hash_table[1][value2]+(sign2*1)
  count_hash_table[2][value3] = count_hash_table[2][value3]+(sign3*1)
  #End Count Sketch

  line = file.readline()
  
  #Query on Stream
  if (count in [100,1000,10000,100000,1000000,10000000,100000000,200000000]):
    print("************************************************************Query",temp_count,"************************************************************")
    summary(count,unique_data_count,n_estimated_morris,n_estimated_fm)
    count_summary(unique_data_count,hash_table,count_hash_table)
    temp_count+=1


file.close()


************************************************************Query 1 ************************************************************
  Exact Element Count    Approximate Element Count using Morris    Exact Distinct Element Count    Approximate Distinct elements count using FM
---------------------  ----------------------------------------  ------------------------------  ----------------------------------------------
                  100                                        63                              98                                              32

Frequency of each distinct Element in the stream {'num':count}
{'298': 1, '5140': 1, '223': 1, '8167': 1, '8115': 1, '4619': 1, '433': 1, '302': 1, '3283': 1, '180': 1, '454': 1, '372': 1, '2496': 1, '256': 1, '4151': 1, '3807': 1, '5067': 1, '4690': 1, '423': 1, '7682': 1, '2402': 1, '1266': 2, '5633': 1, '364': 1, '2705': 1, '138': 1, '2297': 1, '3169': 1, '470': 1, '1483': 1, '6638': 1, '762': 1, '2681': 1, '408': 1, '5648': 1, '44

In [ ]:
summary(count,unique_data_count,n_estimated_morris,n_estimated_fm)
count_summary(unique_data_count,hash_table,count_hash_table)

  Exact Element Count    Approximate Element Count using Morris    Exact Distinct Element Count    Approximate Distinct elements count using FM
---------------------  ----------------------------------------  ------------------------------  ----------------------------------------------
            275732974                                 268435455                           10000                                            8192

Frequency of each distinct Element in the stream {'num':count}
{'298': 70393, '5140': 20633, '223': 70463, '8167': 6237, '8115': 6455, '4619': 23593, '433': 70438, '302': 70284, '3283': 34221, '180': 70656, '454': 70063, '372': 70865, '2496': 42262, '256': 70450, '4151': 26846, '3807': 29512, '5067': 20961, '4690': 22949, '423': 70793, '7682': 7995, '2402': 43706, '1266': 63362, '5633': 17528, '364': 70700, '2705': 39695, '138': 70602, '2297': 45227, '3169': 34910, '470': 71128, '1483': 58776, '6638': 12475, '762': 70004, '2681': 40112, '408': 70625, '5648': 17